# Linear Network

The `nengolib.LinearNetwork` class is a `nengo` network that abstracts away many of the details of Principle 3 from the NEF. Simply supply a `LinearSystem` that you would like to simulate, and the network will map its dynamics onto a recurrently connected ensemble using a given `synapse`. This can be understood equivalently as convolving a causal filter with some arbitrary input signal. 

### Delay Example

For example, the `nengolib.synapses.PadeDelay` is a linear system that we can build into a biologically plausible population of neurons to delay an input signal by some fixed amount of time.

In [ ]:
%pylab inline
import pylab
try:
    import seaborn as sns  # optional; prettier graphs
except ImportError:
    pass

import numpy as np
import nengo
import nengolib
from nengolib.synapses import PadeDelay

delay = 0.1
T = 2.0
dt = 0.001

with nengolib.Network() as model:
    stim = nengo.Node(output=nengo.processes.WhiteSignal(T, high=10))
    
    # Build a LinearNetwork that approximations a delay
    subnet = nengolib.networks.LinearNetwork(
        PadeDelay(3, 4, delay), n_neurons=200, synapse=0.02,
        radii=1.5, dt=dt)
    nengo.Connection(stim, subnet.input, synapse=None)

    # Add some probes
    p = nengo.Probe(subnet.output, synapse=0.01)
    p_stim = nengo.Probe(stim, synapse=0.01)
    
sim = nengo.Simulator(model, dt=dt)
sim.run(T)

And we can visualize the difference between the ideal shifted input and the actual output of the network to see how accurately it performs. 

In [ ]:
offset = int(delay / dt + 1)

pylab.figure()
pylab.title('Delayed Input Signal')
pylab.plot(sim.trange()[offset:], sim.data[p][offset:], label="Actual")
pylab.plot(sim.trange()[offset:], sim.data[p_stim][:-offset], label="Expected")
pylab.legend()
pylab.xlabel('Time (s)')
pylab.show()

### Setting Radii

One of the most subtle but difficult aspects of building these networks is making sure that the representational range that the neurons are optimized over (the eval points and the radius in Nengo), matches the states that the network will actually visit.

Crucially, this depends on how characteristics of the input signal relate to the given dynamics. By default, the `LinearNetwork` class plans for the worst-case input within the given radius, and accordingly plays safe by over-estimating the range of states that the neurons should represent.

If we have more information about the problem, say by having looked at the responses of the states for typical inputs, we may use that to shrink the radii.

In [ ]:
with model:
    p_x = nengo.Probe(subnet.x.output, synapse=0.01)

sim = nengo.Simulator(model, dt=dt)
sim.run(T)

pylab.figure()
pylab.title('States')
pylab.plot(sim.trange(), sim.data[p_x])
pylab.xlabel('Time (s)')
pylab.show()

radii = np.max(abs(sim.data[p_x]), axis=0)
print radii

More information on setting radii will be added here in the future, as well as simulating networks with other synapses on both analog and digital hardware.

### Input and Output Filtering

By default the network will filter the input signal with the same synapse as the recurrent connection. This is how an LTI system is formulated, and so we do that here as well. Thus the _unfiltered_ output from the network is the desired system convolved with the input signal. 

However, if the input signal has already been filtered, and/or we would like the _filtered_ output to correspond to the desired system, then you should pass __`input_synapse=None`__ as an argument. This will avoid filtering the input, in effect giving an output that is a deconvolved version of the desired signal. Filtering this output with the same time-constant will then implement the desired system. But it is important to note that this only holds if the transfer function is strictly proper (it has a zero passthrough $D$ in state-space), otherwise the passthrough component will erroneously have a filter.